In [1]:
# confirm high-RAM runtime configuration
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 34.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [2]:
# confirm usage of GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

zsh:1: command not found: nvidia-smi


In [3]:
import spacy

/Users/ellabettison/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!pip install -U spacy

In [5]:
# download pre-trained RoBERTa transformer model to fine-tune
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [6]:
# install GPU-compatible version of spacy
!pip install spacy[cuda100]==3.2.0

zsh:1: no matches found: spacy[cuda100]==3.2.0


In [7]:
!pip install spacy-transformers

In [8]:
!python -m transformers download nlpaueb/legal-bert-base-uncased

/Users/ellabettison/.pyenv/versions/3.10.0/bin/python: No module named transformers.__main__; 'transformers' is a package and cannot be directly executed


In [9]:
# from google.colab import drive
#drive.mount('/content/drive')

In [10]:
# auto fill config from base config
#!python -m spacy init fill-config drive/MyDrive/base_config.cfg drive/MyDrive/config.cfg

# download config file config.cfg
# delete file from workspace 'content'
# manually update max_epochs in the file from 0 to 90
# upload file

# copy config file from MyDrive
#!cp /content/drive/MyDrive/config.cfg /content/

In [11]:
import time
start = time.time()
# starting at 15:18
# (~2.5hrs tot)

In [37]:
!python -m spacy train ../spacy_data/config.cfg --output ./output --paths.train ../spacy_data/train.spacy --paths.dev ../spacy_data/dev.spacy


ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-10 10:34:39,488] [INFO] Set up nlp object from config
[2022-10-10 10:34:39,508] [INFO] Pipeline: ['tok2vec', 'transformer', 'spancat']
[2022-10-10 10:34:39,508] [INFO] Resuming training for: ['tok2vec']
[2022-10-10 10:34:39,516] [INFO] Created vocabulary
[2022-10-10 10:34:39,518] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another t

In [13]:
#!python -m spacy train /content/drive/MyDrive/spacy_data/config.cfg --output ./output --paths.train drive/MyDrive/spacy_data/train.spacy --paths.dev drive/MyDrive/spacy_data/dev.spacy
# --gpu-id 0

In [14]:
end = time.time()
t_time = end - start
print('training time: {}hrs'.format(round(t_time/3600, 1)))

training time: 0.0hrs


In [15]:
# move best and/or last model to MyDrive
#!cp -r /content/output/model-best /content/drive/MyDrive/
#!cp -r /content/output/model-last /content/drive/MyDrive/

In [16]:
!cp -r output/model-best /
!cp -r output/model-last /

cp: /model-best: Read-only file system
cp: output/model-best: unable to copy extended attributes to /model-best: Read-only file system
cp: /model-best/tokenizer: No such file or directory
cp: /model-best/transformer: No such file or directory
cp: output/model-best/transformer: unable to copy extended attributes to /model-best/transformer: No such file or directory
cp: /model-best/transformer/cfg: No such file or directory
cp: /model-best/transformer/model: No such file or directory
cp: /model-best/vocab: No such file or directory
cp: output/model-best/vocab: unable to copy extended attributes to /model-best/vocab: No such file or directory
cp: /model-best/vocab/vectors: No such file or directory
cp: /model-best/vocab/lookups.bin: No such file or directory
cp: /model-best/vocab/strings.json: No such file or directory
cp: /model-best/vocab/key2row: No such file or directory
cp: /model-best/vocab/vectors.cfg: No such file or directory
cp: /model-best/config.cfg: No such file or directory


In [17]:
!python -m spacy evaluate output/model-best ../spacy_data/dev.spacy

ℹ Using CPU
/Users/ellabettison/.pyenv/versions/3.10.0/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')

================================== Results ==================================

TOK      100.00
SPAN P   84.89 
SPAN R   79.74 
SPAN F   82.24 
SPEED    69    


============================== SPANS (per type) ==============================

           P        R        F
ID     81.60    87.55    84.47
ACR    95.35    80.39    87.23
IND    77.78    70.79    74.12
STP   100.00   100.00   100.00
LEG    95.74    93.75    94.74
LOC    90.20    77.97    83.64
PER    54.55    31.58    40.00
STR   100.00    88.89    94.12
ORG    88.64    76.47    82.11
NAT    85.71   100.00    92.31
SCP   100.00    80.00    88.89
COL   100.00    33.33    50.00
MOD   100.00    50.00    66.67



In [18]:
# get evaluation metrics from BEST MODEL:
#!python -m spacy evaluate /content/drive/MyDrive/model-best drive/MyDrive/spacy_data/dev.spacy

In [19]:
# --------------------------------------------------------------------------------------------------------------------

In [20]:
!python --version

Python 3.10.0


In [21]:
import sysconfig
sysconfig.get_platform()

'macosx-10.15-x86_64'

In [22]:
# copy prodigy wheel files to session contents
!cp /content/drive/MyDrive/linux_prodigy_wheels/ -r /content/

usage: cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file target_file
       cp [-R [-H | -L | -P]] [-fi | -n] [-apvXc] source_file ... target_directory


In [23]:
!pip install linux_prodigy_wheels/prodigy-1.11.7-cp37-cp37m-linux_x86_64.whl

ERROR: prodigy-1.11.7-cp37-cp37m-linux_x86_64.whl is not a supported wheel on this platform.


In [24]:
pip show prodigy

Name: prodigy
Version: 1.11.7
Summary: Radically efficient machine teaching.
Home-page: https://prodi.gy
Author: Explosion
Author-email: contact@explosion.ai
License: All rights reserved
Location: /Users/ellabettison/.pyenv/versions/3.10.0/lib/python3.10/site-packages
Requires: aiofiles, cachetools, catalogue, fastapi, peewee, plac, pydantic, pyjwt, requests, spacy, srsly, toolz, uvicorn, wasabi
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [25]:
# copy prodigy dataset to session contents
!cp /content/drive/MyDrive/jobs_hyphen_ext.jsonl /content/

cp: directory /content does not exist


In [26]:
!prodigy stats -l


============================== ✨  Prodigy Stats ==============================

Version          1.11.7                        
Location         /Users/ellabettison/.pyenv/versions/3.10.0/lib/python3.10/site-packages/prodigy
Prodigy Home     /Users/ellabettison/.prodigy  
Platform         macOS-10.15.7-x86_64-i386-64bit
Python Version   3.10.0                        
Database Name    SQLite                        
Database Id      sqlite                        
Total Datasets   9                             
Total Sessions   40                            


================================ ✨  Datasets ================================

company_names_tokenised_prodigy, company_names_tokenised,
company_names_tokenised_spanish, company_names_tokenised_review,
company_names_reviewed, company_pairs_tokenised, company_names_auto,
company_pairs_auto, company_names_didemo_auto



In [27]:
# !prodigy db-in jobs_re_reviewed jobs_re_reviewed.jsonl
!prodigy db-in hyphen_extended_2 jobs_hyphen_ext.jsonl


✘ Not a valid input file
jobs_hyphen_ext.jsonl



In [28]:
!prodigy stats -l


============================== ✨  Prodigy Stats ==============================

Version          1.11.7                        
Location         /Users/ellabettison/.pyenv/versions/3.10.0/lib/python3.10/site-packages/prodigy
Prodigy Home     /Users/ellabettison/.prodigy  
Platform         macOS-10.15.7-x86_64-i386-64bit
Python Version   3.10.0                        
Database Name    SQLite                        
Database Id      sqlite                        
Total Datasets   9                             
Total Sessions   40                            


================================ ✨  Datasets ================================

company_names_tokenised_prodigy, company_names_tokenised,
company_names_tokenised_spanish, company_names_tokenised_review,
company_names_reviewed, company_pairs_tokenised, company_names_auto,
company_pairs_auto, company_names_didemo_auto



In [29]:
!pip install spacy==3.1.1

  Using cached spacy-3.1.1-cp310-cp310-macosx_10_15_x86_64.whl
  Using cached thinc-8.0.17-cp310-cp310-macosx_10_9_x86_64.whl (648 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.2
    Uninstalling thinc-8.1.2:
      Successfully uninstalled thinc-8.1.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.1.7 requires spacy<4.0.0,>=3.1.3, but you have spacy 3.1.1 which is incompatible.
en-core-web-trf 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.1.1 which is incompatible.
en-core-web-sm 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.1.1 which is incompatible.
en-core-web-lg 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.1.1 which is incompa

In [30]:
!pip install spacy-transformers

  Using cached spacy-3.4.1-cp310-cp310-macosx_10_9_x86_64.whl (6.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.5/759.5 kB 2.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.1.1
    Uninstalling spacy-3.1.1:
      Successfully uninstalled spacy-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prodigy 1.11.7 requires spacy<3.3.0,>=3.1.1, but you have spacy 3.4.1 which is incompatible.


In [31]:
!python -m spacy download en_core_web_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━ 335.6/460.3 MB 12.7 MB/s eta 0:00:10^C
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━ 336.4/460.3 MB 12.5 MB/s eta 0:00:10
ERROR: Operation cancelled by user

Aborted!


In [ ]:
!pip install plotext==4.2.0

In [ ]:
import torch

/Users/ellabettison/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch.cuda.current_device()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
start = time.time()

In [ ]:
!prodigy stats hyphen_extended_2

In [ ]:
!prodigy train-curve --ner hyphen_extended_2 --gpu-id 0 --show-plot
# --training.max_epochs=50 --training.max_steps=20000 --config config.cfg -m en_core_web_trf

In [ ]:
end = time.time()
t_time = end - start
print('train-curve time:', t_time)